In [1]:
import util

In [2]:
%%time
arr = util.get_graph()

CPU times: user 53.3 s, sys: 1min 14s, total: 2min 7s
Wall time: 3min 50s


In [3]:
from queue import Queue
from copy import copy
from typing import List

BREADTH_CUTOFF = 5

def bfs_no_path(src, dst):    
    visited = set()

    q = Queue()
    q.put(src)

    breadth = 0
    while not q.empty():
        print('breadth, qsize:', breadth, q.qsize())
        for _ in range(q.qsize()):
            offset = q.get()
            if offset not in visited:
                visited.add(offset)
                if offset == dst:
                    print('Path of length %s found' % breadth)
                    return
                # add neighbors to queue
                ind = offset // 4
                num_links = arr[ind + 1]
                link_start_ind = ind + 4
                for i in range(link_start_ind, link_start_ind + num_links):
                    q.put(arr[i])
        
        breadth += 1
        if breadth > BREADTH_CUTOFF:
            print('Breadth cutoff exceeded')
            return
    print('No path found')


class Node:
    def __init__(self, offset: int, parent: 'Node', children: 'List[Node]' = []):
        self.offset = offset
        self.parent = parent
        self.children = children
        

def trace_node(node: Node):
    path = []
    
    path.append(node.offset)
    while node.parent:
        node = node.parent
        path.append(node.offset)
    
    return path[::-1]


def bfs_with_path_v0(src, dst):    
    visited = set()

    q = Queue()
    start_node = Node(src, None)
    q.put(start_node)

    visited.add(src)

    breadth = 0
    while not q.empty():
        print('breadth, qsize:', breadth, q.qsize())
        for _ in range(q.qsize()):
            node = q.get()
            offset = node.offset
            
            visited.add(offset)
            if offset == dst:
                print('Path of length %s found' % breadth)
                return trace_node(node)
            # add neighbors to queue
            ind = offset // 4
            num_links = arr[ind + 1]
            link_start_ind = ind + 4
            for i in range(link_start_ind, link_start_ind + num_links):
                to_visit_offset = arr[i]
                if to_visit_offset not in visited:
                    visited.add(to_visit_offset)
                    to_visit_node = Node(to_visit_offset, node)
                    node.children.append(to_visit_node)
                    q.put(to_visit_node)

        breadth += 1
        if breadth > BREADTH_CUTOFF:
            print('Breadth cutoff exceeded')
            return []
    print('No path found')

In [4]:
def bfs(src: int, dst: int) -> int:  
    assert src != dst

    q = Queue()
    q.put(src)

    # mark start node with parent = -1
    arr[src // 4] = -1

    breadth = 1
    while not q.empty():
        print('breadth, qsize:', breadth, q.qsize())
        for _ in range(q.qsize()):
            offset = q.get()
            ind = offset // 4
            num_links = arr[ind + 1]
            link_start_ind = ind + 4
            for i in range(link_start_ind, link_start_ind + num_links):
                to_visit_offset = arr[i]
                # if a node has not had its parent set, it hasn't been visited
                if arr[to_visit_offset // 4] == 0:
                    # mark parent
                    arr[to_visit_offset // 4] = offset
                    if to_visit_offset == dst:
                        print('Path of length %s found' % (breadth + 1))
                        return to_visit_offset
                    # add to queue
                    q.put(to_visit_offset)

        breadth += 1

    print('No path found')

In [5]:
# source and destination, in byte offsets
src = 'Jeff Bezos' # 'Craig Evans (Australian footballer)'
dst = 'Illuminati' # 'Calliandra conferta'

src_offset = util.title_to_offset(src)
dst_offset = util.title_to_offset(dst)

if src_offset is None:
    raise Exception('Source does not exist')
if dst_offset is None:
    raise Exception('Destination does not exist')
    
src_offset, dst_offset

(30406060, 283795788)

In [6]:
%%time
util.clear_parent_refs(arr)

CPU times: user 3.76 s, sys: 3.58 s, total: 7.34 s
Wall time: 11.4 s


In [7]:
%%time
offset = bfs(src_offset, dst_offset)
path = util.trace_path(arr, offset)
[util.offset_to_title(x) for x in path]

breadth, qsize: 1 1
breadth, qsize: 2 153
breadth, qsize: 3 15335
Path of length 4 found
CPU times: user 3.47 s, sys: 1.32 s, total: 4.79 s
Wall time: 7.03 s


['Jeff Bezos',
 'University of Florida',
 'Fraternities and sororities',
 'Illuminati']

In [8]:
[util.offset_to_title(x) for x in [515901088, 141240020, 17305252, 783775484, 219527324, 276863436, 586209616]]

['Calliandra conferta',
 'Calliandra',
 'Fabaceae',
 'Acacia',
 'Australia',
 'Grovedale Football Club',
 'Craig Evans (Australian footballer)']